# Recurrent neural networks for text generation

In this notebook, we will build a character-wise RNN trained on Anna Karenina. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([65, 78, 44, 60, 34, 54, 37,  6,  8, 81, 81, 81, 16, 44, 60, 60, 49,
        6,  2, 44, 23,  4,  0,  4, 54, 74,  6, 44, 37, 54,  6, 44,  0,  0,
        6, 44,  0,  4, 80, 54,  5,  6, 54, 50, 54, 37, 49,  6, 58, 55, 78,
       44, 60, 60, 49,  6,  2, 44, 23,  4,  0, 49,  6,  4, 74,  6, 58, 55,
       78, 44, 60, 60, 49,  6,  4, 55,  6,  4, 34, 74,  6, 75, 24, 55, 81,
       24, 44, 49, 70, 81, 81, 66, 50, 54, 37, 49, 34, 78,  4, 55], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the batch size and number of batches we can make
    batch_size = n_seqs * n_steps 
    n_batches =  len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr =  arr[:n_batches*batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs,-1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:,n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros(x.shape)
        y[:,:-1],y[:,-1] = x[:,1:] ,x[:,0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[65 78 44 60 34 54 37  6  8 81]
 [ 6 44 23  6 55 75 34  6 31 75]
 [50  4 55 70 81 81 35 40 54 74]
 [55  6 43 58 37  4 55 31  6 78]
 [ 6  4 34  6  4 74 10  6 74  4]
 [ 6  7 34  6 24 44 74 81 75 55]
 [78 54 55  6 82 75 23 54  6  2]
 [ 5  6 15 58 34  6 55 75 24  6]
 [34  6  4 74 55 67 34 70  6 11]
 [ 6 74 44  4 43  6 34 75  6 78]]

y
 [[ 78.  44.  60.  34.  54.  37.   6.   8.  81.  81.]
 [ 44.  23.   6.  55.  75.  34.   6.  31.  75.   4.]
 [  4.  55.  70.  81.  81.  35.  40.  54.  74.  10.]
 [  6.  43.  58.  37.   4.  55.  31.   6.  78.   4.]
 [  4.  34.   6.   4.  74.  10.   6.  74.   4.  37.]
 [  7.  34.   6.  24.  44.  74.  81.  75.  55.   0.]
 [ 54.  55.   6.  82.  75.  23.  54.   6.   2.  75.]
 [  6.  15.  58.  34.   6.  55.  75.  24.   6.  74.]
 [  6.   4.  74.  55.  67.  34.  70.   6.  11.  78.]
 [ 74.  44.   4.  43.   6.  34.  75.   6.  78.  54.]]


In [9]:
x

array([[65, 78, 44, 60, 34, 54, 37,  6,  8, 81, 81, 81, 16, 44, 60, 60, 49,
         6,  2, 44, 23,  4,  0,  4, 54, 74,  6, 44, 37, 54,  6, 44,  0,  0,
         6, 44,  0,  4, 80, 54,  5,  6, 54, 50, 54, 37, 49,  6, 58, 55],
       [ 6, 44, 23,  6, 55, 75, 34,  6, 31, 75,  4, 55, 31,  6, 34, 75,  6,
        74, 34, 44, 49, 10, 35,  6, 44, 55, 74, 24, 54, 37, 54, 43,  6, 62,
        55, 55, 44, 10,  6, 74, 23,  4,  0,  4, 55, 31, 10,  6, 15, 58],
       [50,  4, 55, 70, 81, 81, 35, 40, 54, 74, 10,  6,  4, 34, 67, 74,  6,
        74, 54, 34, 34,  0, 54, 43, 70,  6, 11, 78, 54,  6, 60, 37,  4, 82,
        54,  6,  4, 74,  6, 23, 44, 31, 55,  4,  2,  4, 82, 54, 55, 34],
       [55,  6, 43, 58, 37,  4, 55, 31,  6, 78,  4, 74,  6, 82, 75, 55, 50,
        54, 37, 74, 44, 34,  4, 75, 55,  6, 24,  4, 34, 78,  6, 78,  4, 74,
        81, 15, 37, 75, 34, 78, 54, 37,  6, 24, 44, 74,  6, 34, 78,  4],
       [ 6,  4, 34,  6,  4, 74, 10,  6, 74,  4, 37, 36, 35,  6, 74, 44,  4,
        43,  6, 34, 78, 

In [10]:
y

array([[ 78.,  44.,  60.,  34.,  54.,  37.,   6.,   8.,  81.,  81.,  81.,
         16.,  44.,  60.,  60.,  49.,   6.,   2.,  44.,  23.,   4.,   0.,
          4.,  54.,  74.,   6.,  44.,  37.,  54.,   6.,  44.,   0.,   0.,
          6.,  44.,   0.,   4.,  80.,  54.,   5.,   6.,  54.,  50.,  54.,
         37.,  49.,   6.,  58.,  55.,  65.],
       [ 44.,  23.,   6.,  55.,  75.,  34.,   6.,  31.,  75.,   4.,  55.,
         31.,   6.,  34.,  75.,   6.,  74.,  34.,  44.,  49.,  10.,  35.,
          6.,  44.,  55.,  74.,  24.,  54.,  37.,  54.,  43.,   6.,  62.,
         55.,  55.,  44.,  10.,   6.,  74.,  23.,   4.,   0.,   4.,  55.,
         31.,  10.,   6.,  15.,  58.,   6.],
       [  4.,  55.,  70.,  81.,  81.,  35.,  40.,  54.,  74.,  10.,   6.,
          4.,  34.,  67.,  74.,   6.,  74.,  54.,  34.,  34.,   0.,  54.,
         43.,  70.,   6.,  11.,  78.,  54.,   6.,  60.,  37.,   4.,  82.,
         54.,   6.,   4.,  74.,   6.,  23.,  44.,  31.,  55.,   4.,   2.,
          4.,  82.,  5

If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [11]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32,[batch_size,num_steps],name="inputs")
    targets = tf.placeholder(tf.int32,[batch_size,num_steps],name="targets")
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32,name="keep_prob")
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. For example,

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow will create different weight matrices for all `cell` objects. Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

> **Exercise:** Below, implement the `build_lstm` function to create these LSTM cells and the initial state.

In [12]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    # Add dropout to the cell outputs
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper( tf.contrib.rnn.BasicLSTMCell(lstm_size)) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size,tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [13]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output,axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output,[-1,in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size),stddev=0.1))
        softmax_b = tf.Variable(tf.zeros([out_size]))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits =  tf.add(tf.matmul(x,softmax_w),softmax_b) 
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits,name ="out")
    print(out)
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [14]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets,num_classes)
    y_reshaped =  tf.reshape(y_one_hot,logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_reshaped))
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [15]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [16]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size,num_steps)
        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size,num_layers,batch_size,self.keep_prob)
        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs,num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell,x_one_hot,initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs,lstm_size,num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits,self.targets,lstm_size,num_classes)
        self.optimizer = build_optimizer(self.loss,learning_rate,grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [85]:
batch_size = 100         # Sequences per batch
num_steps = 150          # Number of sequence steps per batch
lstm_size = 550         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [86]:
epochs = 40
# Save every N iterations
save_every_n = 200
with tf.device("/gpu:0"):
    model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Tensor("out:0", shape=(15000, 83), dtype=float32)
Epoch: 1/40...  Training Step: 1...  Training loss: 4.4183...  0.3233 sec/batch
Epoch: 1/40...  Training Step: 2...  Training loss: 4.3311...  0.2659 sec/batch
Epoch: 1/40...  Training Step: 3...  Training loss: 3.7770...  0.2862 sec/batch
Epoch: 1/40...  Training Step: 4...  Training loss: 4.8723...  0.2633 sec/batch
Epoch: 1/40...  Training Step: 5...  Training loss: 4.0427...  0.2822 sec/batch
Epoch: 1/40...  Training Step: 6...  Training loss: 3.8744...  0.2668 sec/batch
Epoch: 1/40...  Training Step: 7...  Training loss: 3.6178...  0.2791 sec/batch
Epoch: 1/40...  Training Step: 8...  Training loss: 3.3741...  0.2707 sec/batch
Epoch: 1/40...  Training Step: 9...  Training loss: 3.2951...  0.2888 sec/batch
Epoch: 1/40...  Training Step: 10...  Training loss: 3.2585...  0.2624 sec/batch
Epoch: 1/40...  Training Step: 11...  Training loss: 3.2080...  0.2661 sec/batch
Epoch: 1/40...  Training Step: 12...  Training loss: 3.1941...  0.28

Epoch: 1/40...  Training Step: 102...  Training loss: 3.0772...  0.2631 sec/batch
Epoch: 1/40...  Training Step: 103...  Training loss: 3.0546...  0.2655 sec/batch
Epoch: 1/40...  Training Step: 104...  Training loss: 3.0536...  0.2630 sec/batch
Epoch: 1/40...  Training Step: 105...  Training loss: 3.0540...  0.2663 sec/batch
Epoch: 1/40...  Training Step: 106...  Training loss: 3.0260...  0.2776 sec/batch
Epoch: 1/40...  Training Step: 107...  Training loss: 3.0397...  0.2811 sec/batch
Epoch: 1/40...  Training Step: 108...  Training loss: 3.0197...  0.2761 sec/batch
Epoch: 1/40...  Training Step: 109...  Training loss: 3.0122...  0.2710 sec/batch
Epoch: 1/40...  Training Step: 110...  Training loss: 3.0285...  0.2667 sec/batch
Epoch: 1/40...  Training Step: 111...  Training loss: 3.0085...  0.2619 sec/batch
Epoch: 1/40...  Training Step: 112...  Training loss: 3.0200...  0.2672 sec/batch
Epoch: 1/40...  Training Step: 113...  Training loss: 3.0013...  0.2756 sec/batch
Epoch: 1/40...  

Epoch: 2/40...  Training Step: 201...  Training loss: 2.3864...  0.2629 sec/batch
Epoch: 2/40...  Training Step: 202...  Training loss: 2.4106...  0.2618 sec/batch
Epoch: 2/40...  Training Step: 203...  Training loss: 2.3991...  0.2621 sec/batch
Epoch: 2/40...  Training Step: 204...  Training loss: 2.4209...  0.2616 sec/batch
Epoch: 2/40...  Training Step: 205...  Training loss: 2.3880...  0.2623 sec/batch
Epoch: 2/40...  Training Step: 206...  Training loss: 2.3954...  0.2622 sec/batch
Epoch: 2/40...  Training Step: 207...  Training loss: 2.4127...  0.2617 sec/batch
Epoch: 2/40...  Training Step: 208...  Training loss: 2.3709...  0.2621 sec/batch
Epoch: 2/40...  Training Step: 209...  Training loss: 2.3769...  0.2649 sec/batch
Epoch: 2/40...  Training Step: 210...  Training loss: 2.3971...  0.2632 sec/batch
Epoch: 2/40...  Training Step: 211...  Training loss: 2.4110...  0.2630 sec/batch
Epoch: 2/40...  Training Step: 212...  Training loss: 2.3960...  0.2620 sec/batch
Epoch: 2/40...  

Epoch: 3/40...  Training Step: 301...  Training loss: 2.1784...  0.2630 sec/batch
Epoch: 3/40...  Training Step: 302...  Training loss: 2.1653...  0.2627 sec/batch
Epoch: 3/40...  Training Step: 303...  Training loss: 2.1557...  0.2626 sec/batch
Epoch: 3/40...  Training Step: 304...  Training loss: 2.1712...  0.2624 sec/batch
Epoch: 3/40...  Training Step: 305...  Training loss: 2.1647...  0.2633 sec/batch
Epoch: 3/40...  Training Step: 306...  Training loss: 2.1863...  0.2622 sec/batch
Epoch: 3/40...  Training Step: 307...  Training loss: 2.1579...  0.2630 sec/batch
Epoch: 3/40...  Training Step: 308...  Training loss: 2.1632...  0.2621 sec/batch
Epoch: 3/40...  Training Step: 309...  Training loss: 2.1376...  0.2626 sec/batch
Epoch: 3/40...  Training Step: 310...  Training loss: 2.1353...  0.2620 sec/batch
Epoch: 3/40...  Training Step: 311...  Training loss: 2.1509...  0.2623 sec/batch
Epoch: 3/40...  Training Step: 312...  Training loss: 2.1651...  0.2625 sec/batch
Epoch: 3/40...  

Epoch: 4/40...  Training Step: 401...  Training loss: 1.9656...  0.2627 sec/batch
Epoch: 4/40...  Training Step: 402...  Training loss: 1.9980...  0.2625 sec/batch
Epoch: 4/40...  Training Step: 403...  Training loss: 1.9679...  0.2627 sec/batch
Epoch: 4/40...  Training Step: 404...  Training loss: 1.9291...  0.2620 sec/batch
Epoch: 4/40...  Training Step: 405...  Training loss: 1.9821...  0.2631 sec/batch
Epoch: 4/40...  Training Step: 406...  Training loss: 1.9676...  0.2628 sec/batch
Epoch: 4/40...  Training Step: 407...  Training loss: 1.9343...  0.2623 sec/batch
Epoch: 4/40...  Training Step: 408...  Training loss: 1.9912...  0.2630 sec/batch
Epoch: 4/40...  Training Step: 409...  Training loss: 1.9566...  0.2639 sec/batch
Epoch: 4/40...  Training Step: 410...  Training loss: 1.9595...  0.2636 sec/batch
Epoch: 4/40...  Training Step: 411...  Training loss: 1.9882...  0.2624 sec/batch
Epoch: 4/40...  Training Step: 412...  Training loss: 1.9373...  0.2630 sec/batch
Epoch: 4/40...  

...

Epoch: 38/40...  Training Step: 4887...  Training loss: 0.8809...  0.2648 sec/batch
Epoch: 38/40...  Training Step: 4888...  Training loss: 0.8391...  0.2691 sec/batch
Epoch: 38/40...  Training Step: 4889...  Training loss: 0.8496...  0.2659 sec/batch
Epoch: 38/40...  Training Step: 4890...  Training loss: 0.8484...  0.2685 sec/batch
Epoch: 38/40...  Training Step: 4891...  Training loss: 0.8440...  0.2660 sec/batch
Epoch: 38/40...  Training Step: 4892...  Training loss: 0.8543...  0.2630 sec/batch
Epoch: 38/40...  Training Step: 4893...  Training loss: 0.8590...  0.2653 sec/batch
Epoch: 38/40...  Training Step: 4894...  Training loss: 0.8410...  0.2637 sec/batch
Epoch: 38/40...  Training Step: 4895...  Training loss: 0.8318...  0.2643 sec/batch
Epoch: 38/40...  Training Step: 4896...  Training loss: 0.8569...  0.2665 sec/batch
Epoch: 38/40...  Training Step: 4897...  Training loss: 0.8538...  0.2689 sec/batch
Epoch: 38/40...  Training Step: 4898...  Training loss: 0.8691...  0.2652 se

Epoch: 38/40...  Training Step: 4985...  Training loss: 0.8420...  0.2670 sec/batch
Epoch: 38/40...  Training Step: 4986...  Training loss: 0.8562...  0.2728 sec/batch
Epoch: 38/40...  Training Step: 4987...  Training loss: 0.8577...  0.2755 sec/batch
Epoch: 38/40...  Training Step: 4988...  Training loss: 0.8610...  0.2719 sec/batch
Epoch: 38/40...  Training Step: 4989...  Training loss: 0.8665...  0.2632 sec/batch
Epoch: 38/40...  Training Step: 4990...  Training loss: 0.8497...  0.2631 sec/batch
Epoch: 38/40...  Training Step: 4991...  Training loss: 0.8699...  0.2645 sec/batch
Epoch: 38/40...  Training Step: 4992...  Training loss: 0.8577...  0.2641 sec/batch
Epoch: 38/40...  Training Step: 4993...  Training loss: 0.8442...  0.2649 sec/batch
Epoch: 38/40...  Training Step: 4994...  Training loss: 0.8365...  0.2640 sec/batch
Epoch: 38/40...  Training Step: 4995...  Training loss: 0.8577...  0.2643 sec/batch
Epoch: 38/40...  Training Step: 4996...  Training loss: 0.8800...  0.2633 se

Epoch: 39/40...  Training Step: 5081...  Training loss: 0.8480...  0.2641 sec/batch
Epoch: 39/40...  Training Step: 5082...  Training loss: 0.8432...  0.2633 sec/batch
Epoch: 39/40...  Training Step: 5083...  Training loss: 0.8513...  0.2650 sec/batch
Epoch: 39/40...  Training Step: 5084...  Training loss: 0.8479...  0.2631 sec/batch
Epoch: 39/40...  Training Step: 5085...  Training loss: 0.8577...  0.2641 sec/batch
Epoch: 39/40...  Training Step: 5086...  Training loss: 0.8496...  0.2632 sec/batch
Epoch: 39/40...  Training Step: 5087...  Training loss: 0.8341...  0.2642 sec/batch
Epoch: 39/40...  Training Step: 5088...  Training loss: 0.8581...  0.2635 sec/batch
Epoch: 39/40...  Training Step: 5089...  Training loss: 0.8494...  0.2638 sec/batch
Epoch: 39/40...  Training Step: 5090...  Training loss: 0.8441...  0.2643 sec/batch
Epoch: 39/40...  Training Step: 5091...  Training loss: 0.8525...  0.2632 sec/batch
Epoch: 39/40...  Training Step: 5092...  Training loss: 0.8576...  0.2642 se

Epoch: 40/40...  Training Step: 5179...  Training loss: 0.8470...  0.2800 sec/batch
Epoch: 40/40...  Training Step: 5180...  Training loss: 0.8337...  0.2751 sec/batch
Epoch: 40/40...  Training Step: 5181...  Training loss: 0.8456...  0.2675 sec/batch
Epoch: 40/40...  Training Step: 5182...  Training loss: 0.8388...  0.2681 sec/batch
Epoch: 40/40...  Training Step: 5183...  Training loss: 0.8665...  0.2671 sec/batch
Epoch: 40/40...  Training Step: 5184...  Training loss: 0.8508...  0.2667 sec/batch
Epoch: 40/40...  Training Step: 5185...  Training loss: 0.8420...  0.2657 sec/batch
Epoch: 40/40...  Training Step: 5186...  Training loss: 0.8457...  0.2653 sec/batch
Epoch: 40/40...  Training Step: 5187...  Training loss: 0.8370...  0.2667 sec/batch
Epoch: 40/40...  Training Step: 5188...  Training loss: 0.8620...  0.2666 sec/batch
Epoch: 40/40...  Training Step: 5189...  Training loss: 0.8551...  0.2643 sec/batch
Epoch: 40/40...  Training Step: 5190...  Training loss: 0.8553...  0.2648 se

Epoch: 40/40...  Training Step: 5275...  Training loss: 0.8186...  0.2732 sec/batch
Epoch: 40/40...  Training Step: 5276...  Training loss: 0.8097...  0.2660 sec/batch
Epoch: 40/40...  Training Step: 5277...  Training loss: 0.8329...  0.2675 sec/batch
Epoch: 40/40...  Training Step: 5278...  Training loss: 0.8295...  0.2665 sec/batch
Epoch: 40/40...  Training Step: 5279...  Training loss: 0.8453...  0.2661 sec/batch
Epoch: 40/40...  Training Step: 5280...  Training loss: 0.8322...  0.2653 sec/batch
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [87]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i5280_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l550.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l550.ckpt"
all_model_checkpoint_pa

In [88]:
print(tf.__version__)

1.1.0-rc2


## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [89]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [90]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [91]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i5280_l550.ckpt'

In [92]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 83), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i5280_l550.ckpt
Far? Lord, and her
conversation with Anna and of the carriage that in her place, and
followed the stars and painful for the lank of the portrait with which
he was the senertay of marshal ones, was still also at the same point;
and for the old man a woman's process of receiving in at a sense of
tenderness, or that the sen eash thing, he too going to draw his corf
coat that he had a strange resolute, and it was in the world on the
work into the scep chulds of serious and calriage with her, which he
had the same, but with a scrue, at the service when the condition of the word
had been said, as though of him, a gray wheee, he nows whom his hair, as though
he lived in his subdued and ready, and talked towards her, she felt he
saw his wife, and so he did not speak to him, and the soone had been
preven through the same eyes, and a child and standing at work on one
side and still

In [95]:
checkpoint = 'checkpoints/i200_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 83), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i200_l550.ckpt
Farld, the le toung of ald an he war tha serang ho se win teren the sous sarit ton thit the he and. sothes, wetint at on andith taste the heres ot antinn an the tond
an whe th th wint ar ale ad at ha tor wathas angiton ta wale ared at wind the he tered tor woret
ad he hurerang this arere tound sas oot the hith tho his te hor tee har, thisd te th and and tit arthe wans tor wor hir, ant ot tind sot in toris the wind se and sal tan to the torendes, thas

tare so to tho tote wat oun wond sot ho shares ther. "ne
 theresed sand wanthe serad thand sothe sand thit or anges, bat on aner as tisd
the tiled, toris hor tot we wher ale har thet
angithe ho tas oot anthes,e tha sard aste the this hind hot otis on te and wote he wott ant ot hes
tasdis hat hurd ont int ore ant on tar thes
""
""
ses tous,e sad sot thes tard warethe the hat ore has sas and south so to she to tar an at th the 

In [98]:
checkpoint = 'checkpoints/i1000_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 83), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i1000_l550.ckpt
Farrdy, and he has
face and drawing off. They he had now that hel how this waser and to
the days. As it's horse who wanted to the saw in the compesser,
the provest of him. Have to be a face there the balked with
showing his broknon, that had barried had and her the shaless of the hould
book trieds, and he has feeling at the same and, he wents waited
to him, that so pussed to himself hursions, and then wondered the
distrest to talk to her to all as their husband, a drew this take
or help, and he had shouted to believe in the houses, who hasted, andowe
the peasants they he had been stiritian in to see, his
from husband that he showed, shooking to hat all this simple, and
seeching the position and at the folemants of her honses, had answered
angry he desest dable on a bight alont at the trien so the beginess
indenires and histless to the post conversate to herese. But to see

In [99]:
checkpoint = 'checkpoints/i2000_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 83), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i2000_l550.ckpt
Farm, and he was
a misulation on the closer was at all of the bring and sat dewar, and she had
to be busing a good himself to be a splendid man, as a man, was an
affection for the same tasis time, and went in to be rushen. "All the
same thing the meador, there's a polute mas about me. But it sere of the thought
of another, inco precense everything as a marr'alle pass it will be
dressones..."

Along the dows of his work she was the old principriors before him to ask
him in thes, and was a barroraction; bringing to and a shried out with
always wile at the table was beind taking the boots acroad, and short his
baby with two chief tackes to ter or not to the side.

At there instens of his been, and almost right, and so heard, and tried to be
affaired the person, he sawn instantent to the steps; and he felt as he talked
to him, had to set her far his own sight of her. And in t

In [100]:
checkpoint = 'checkpoints/i3000_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 83), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i3000_l550.ckpt
Farm, the friends of his
head, all this she had spoke to his mother while he had to go on to the
povition. Before he had been at these moments which he was after, or an impelle
cloupin of meeting all this were all hus intimate there. Betsy ofe and the peasant
was new away, when he was told the daughter of the cruel that outhider, a
thing, humiliating with pleasure that he was nothing. After a paint
he was nothing to hear that it was all in the most propostion of his
feer, was the more of his opponent. The man bloods over the marehal of
the province he had told him a face, and that through that way the other
peasants were a plan a little or an all man's position, and to this position who
was her out in the might have the peace, showing him and the hildless,
secrntal smile of prainter into the second people she felt that the prince
had been put in a legs, she was always lik

In [101]:
checkpoint = 'checkpoints/i4000_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 83), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i4000_l550.ckpt
Farriand who has
sucred in it from that stung he had not spent him witto the same and
happy, and her face wore a mental fold of reason after an order that
she said: "I am on the little day alth I had not taken in the watch
of my work," she said, and she smiled. He felt a gladn. Helply a mistake
before, as he said in the shrill one as the peasant would go in all the
story--he knew from what in taking the same thing to the meadow,
she could disapprove of it, as he could not conceive a stone, and
seared to see Anna; he had strong to her riddle, and they saw through,
that he had struck him as to the way of a significal idivier of the
elections, began to begue. He dread in a during that they called him out
of the divining embarrassment, while she had firmly desired to
go on such and they shut himself by all about her sis. His so lunges at
the momerase of tenderness, and sturie

In [104]:
checkpoint = 'checkpoints/i5280_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="lord")
print(samp)

Tensor("out:0", shape=(1, 83), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i5280_l550.ckpt
lord for the sake of
the secf and saying something, and so he struck him. "But what does he
love?" said Dolly, and his hat and flushing hew shoulders. "Yes, I shall again
love with this house and have been brought down by the person, that her
companions will have behind him to look at her. He sent a glee for the
chaig and and his part of her surpliss, the subject of comman senince.

"Ah, that was a decoration of daulh at returd for you. I came into
every details, and I cannot live without their feeling and calm hopel.
Than the world is not wanted to talk any disconcentress in which she
would be deceived with..."

"And how do you tried?" he said. "We showed that he's doing how about
it, and so." "Well, and don't bitter from my husband. And so I describe
our her, he won't go and seemed to save her. And the stags of its
dusts..."

"Pardot, cheschair, to be false for that lif